# Predicting Next Fixture Points Based on Past Performance

In [1]:
import pandas as pd
import re
import random
import requests
import pickle

## Get player data and gameweek data

Import packages

Get data from the FPL API

Code source: https://medium.com/analytics-vidhya/getting-started-with-fantasy-premier-league-data-56d3b9be8c32

In [2]:
# set url for fantasy PL API
api_url = "https://fantasy.premierleague.com/api/bootstrap-static/"

# download the webpage
data = requests.get(api_url)

json = data.json()

json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [3]:
# build a dataframe
players = pd.DataFrame(json['elements'])

#players.columns

In [4]:
#select only relevant columns from elements_df, not all needed at this point but maybe useful in the future
#players_df_select = elements_df[['first_name','second_name','team','element_type','selected_by_percent',
#                                'now_cost','minutes','transfers_in','value_season','total_points']]

# use all columns 
players_df_select = players

# combine first and last names to get player full names
players_df_select['full_name'] = players_df_select[['first_name', 'second_name']].agg(' '.join, axis=1)

# drop first and last name columns
players_df_select = players_df_select.drop(['first_name', 'second_name'], axis = 1)

# player prices are 10x the true value. Divide the prices by 10 to get the true values
players_df_select['now_cost'] = players_df_select['now_cost']/10

players_df_select.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,full_name
0,100.0,100.0,58822,0,0,-3,3,0,2,0.5,...,160,481,187,410,154,406,154,0.00,0.00,Cédric Alves Soares
1,NaN,NaN,84450,0,0,0,0,1,3,3.8,...,113,89,33,48,20,100,36,1.06,0.45,Granit Xhaka
2,50.0,100.0,153256,-1,1,-4,4,1,3,0.2,...,313,476,180,383,165,204,62,0.81,0.00,Mohamed Elneny
3,NaN,NaN,156074,0,0,-3,3,0,2,0.8,...,167,326,111,414,158,431,164,0.00,0.00,Rob Holding
4,75.0,100.0,167199,-1,1,-3,3,0,3,2.4,...,187,134,47,124,57,256,83,1.03,0.58,Thomas Partey


In [5]:
# get team info
teams = pd.DataFrame(json['teams'])

teams.head()

,code,draw,form,id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,...,None,False,0,1220,1270,1240,1250,1200,1270,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,...,None,False,0,1090,1100,1110,1130,1090,1110,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,...,None,False,0,1060,1090,1070,1130,1050,1080,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,...,None,False,0,1100,1130,1100,1110,1130,1160,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,...,None,False,0,1150,1160,1140,1160,1170,1190,131


Get team defensive strength for each team

In [6]:
team_strength_def = teams[['id', 'name', 'strength_defence_away', 'strength_defence_home']]

team_strength_def

,id,name,strength_defence_away,strength_defence_home
0,1,Arsenal,1270,1200
1,2,Aston Villa,1110,1090
2,3,Bournemouth,1080,1050
3,4,Brentford,1160,1130
4,5,Brighton,1190,1170
5,6,Chelsea,1210,1180
6,7,Crystal Palace,1140,1080
7,8,Everton,1080,1080
8,9,Fulham,1130,1100
9,10,Leicester,1120,1180


Get team attack strength for each team

In [7]:
team_strength_att = teams[['id', 'name', 'strength_attack_away', 'strength_attack_home']]

team_strength_att

,id,name,strength_attack_away,strength_attack_home
0,1,Arsenal,1250,1240
1,2,Aston Villa,1130,1110
2,3,Bournemouth,1130,1070
3,4,Brentford,1110,1100
4,5,Brighton,1160,1140
5,6,Chelsea,1210,1200
6,7,Crystal Palace,1170,1100
7,8,Everton,1080,1080
8,9,Fulham,1070,1070
9,10,Leicester,1160,1080


Get position info

In [8]:
# get position information from 'element_types'
positions = pd.DataFrame(json['element_types'])

positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],74
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],243
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],313
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],91


Merge the players with their teams

In [9]:
# merge player data with teams and positions
player_team_merge = pd.merge(
    left = players_df_select,
    right = teams,
    left_on = 'team',
    right_on = 'id'
)

player_team_merge[['full_name', 'name']].head()

,full_name,name
0,Cédric Alves Soares,Arsenal
1,Granit Xhaka,Arsenal
2,Mohamed Elneny,Arsenal
3,Rob Holding,Arsenal
4,Thomas Partey,Arsenal


Merge the players with their positions

In [10]:
player_team_pos_merge = pd.merge(
    left = player_team_merge,
    right = positions,
    left_on = 'element_type',
    right_on = 'id'
)

player_team_pos_merge[['full_name', 'name', 'singular_name_short']].head()

,full_name,name,singular_name_short
0,Cédric Alves Soares,Arsenal,DEF
1,Rob Holding,Arsenal,DEF
2,Kieran Tierney,Arsenal,DEF
3,Benjamin White,Arsenal,DEF
4,Takehiro Tomiyasu,Arsenal,DEF


In [11]:
# rename columns
player_team_pos_merge = player_team_pos_merge.rename(
    columns={'name':'team_name', 'singular_name_short':'position_name'}
)

player_team_pos_merge[['full_name', 'team_name', 'position_name']].head()

,full_name,team_name,position_name
0,Cédric Alves Soares,Arsenal,DEF
1,Rob Holding,Arsenal,DEF
2,Kieran Tierney,Arsenal,DEF
3,Benjamin White,Arsenal,DEF
4,Takehiro Tomiyasu,Arsenal,DEF


Get player gameweek data from https://fantasy.premierleague.com/api/element-summary/

In [12]:
# function for getting specific player gameweek history
def get_history(player_id):
    ''' get all gameweek history for a given player'''
    
    # request data from API 
    data = requests.get("https://fantasy.premierleague.com/api/element-summary/" + str(player_id) + "/")
    json = data.json()
    
    # turn data into Pandas dataframe
    df = pd.DataFrame(json['history'])
    
    return df

get_history(1)  

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out
0,1,1,7,0,False,2022-08-05T19:00:00Z,0,2,1,0,...,0,0.00,0.00,0.00,0.00,45,0,23970,0,0
1,1,11,10,0,True,2022-08-13T14:00:00Z,4,2,2,0,...,0,0.00,0.00,0.00,0.00,44,-5169,24193,1361,6530
2,1,21,3,0,False,2022-08-20T16:30:00Z,0,3,3,0,...,0,0.00,0.00,0.00,0.00,44,-4337,20960,879,5216
3,1,31,9,0,True,2022-08-27T16:30:00Z,2,1,4,0,...,0,0.00,0.00,0.00,0.00,43,-2988,18825,577,3565
4,1,41,2,0,True,2022-08-31T18:30:00Z,2,1,5,0,...,0,0.00,0.00,0.00,0.00,43,-1611,17790,405,2016
5,1,56,14,0,False,2022-09-04T15:30:00Z,3,1,6,0,...,0,0.00,0.00,0.00,0.00,42,-1207,17133,516,1723
6,1,72,4,0,False,2022-09-18T11:00:00Z,0,3,8,0,...,0,0.00,0.00,0.00,0.00,42,-445,16837,114,559
7,1,81,18,0,True,2022-10-01T11:30:00Z,3,1,9,0,...,0,0.00,0.00,0.00,0.00,42,-1572,15362,64,1636
8,1,91,12,0,True,2022-10-09T15:30:00Z,3,2,10,0,...,0,0.00,0.00,0.00,0.00,42,-828,14630,148,976
9,1,104,11,0,False,2022-10-16T13:00:00Z,0,1,11,0,...,0,0.00,0.00,0.00,0.00,42,-229,14586,227,456


In [13]:
from tqdm.auto import tqdm
tqdm.pandas()

In [14]:
# join team name
players = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
players = players.merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)

# rename columns
players = players.rename(
    columns={'name':'team', 'singular_name_short':'position'}
)

players.head()


,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,ep_next,ep_this,...,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,full_name,team,position
0,100.0,100.0,58822,0,0,-3,3,0,0.5,0.5,...,187,410,154,406,154,0.00,0.00,Cédric Alves Soares,Arsenal,DEF
1,NaN,NaN,156074,0,0,-3,3,0,0.8,0.8,...,111,414,158,431,164,0.00,0.00,Rob Holding,Arsenal,DEF
2,100.0,100.0,192895,-1,1,-4,4,0,0.8,0.8,...,125,354,134,201,74,0.79,0.20,Kieran Tierney,Arsenal,DEF
3,NaN,NaN,198869,0,0,2,-2,0,5.5,5.5,...,9,37,7,25,9,1.13,0.60,Benjamin White,Arsenal,DEF
4,100.0,100.0,223723,0,0,-3,3,0,2.5,2.5,...,63,311,117,238,88,0.72,0.36,Takehiro Tomiyasu,Arsenal,DEF


In [15]:
# get gameweek history for all players
points = players['id_player'].progress_apply(get_history)

# combine results into one dataframe
points = pd.concat(df for df in points)

# join full_name
points = players[['id_player', 'full_name', 'team', 'position']].merge(
    points,
    left_on='id_player',
    right_on='element'
)

  0%|          | 0/721 [00:00<?, ?it/s]

In [16]:
# merge opponent defensive strength
points = pd.merge(left = points,
                  right = team_strength_def[['id', 'strength_defence_away', 
                                             'strength_defence_home']],
                  how = 'left',
                  left_on = 'opponent_team',
                  right_on = 'id'
).drop(
    'id', axis = 1
).rename(
    columns={'strength_defence_away':'opp_def_strength_away', 'strength_defence_home':'opp_def_strength_home'}
)

points.head()

,id_player,full_name,team,position,element,fixture,opponent_team,total_points,was_home,kickoff_time,...,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out,opp_def_strength_away,opp_def_strength_home
0,1,Cédric Alves Soares,Arsenal,DEF,1,1,7,0,False,2022-08-05T19:00:00Z,...,0.00,0.00,0.00,45,0,23970,0,0,1140,1080
1,1,Cédric Alves Soares,Arsenal,DEF,1,11,10,0,True,2022-08-13T14:00:00Z,...,0.00,0.00,0.00,44,-5169,24193,1361,6530,1120,1180
2,1,Cédric Alves Soares,Arsenal,DEF,1,21,3,0,False,2022-08-20T16:30:00Z,...,0.00,0.00,0.00,44,-4337,20960,879,5216,1080,1050
3,1,Cédric Alves Soares,Arsenal,DEF,1,31,9,0,True,2022-08-27T16:30:00Z,...,0.00,0.00,0.00,43,-2988,18825,577,3565,1130,1100
4,1,Cédric Alves Soares,Arsenal,DEF,1,41,2,0,True,2022-08-31T18:30:00Z,...,0.00,0.00,0.00,43,-1611,17790,405,2016,1110,1090


In [17]:
# assign correct home/away opponent defensive strength for each fixture
def opp_def_strength(row):
    if row['was_home'] == False:
        return row['opp_def_strength_home']
    elif row['was_home'] == True:
        return row['opp_def_strength_away']
    else:
        return "Unknown"

points['opp_def_strength'] = points.apply(lambda row: opp_def_strength(row), axis = 1)

points = points.drop(['opp_def_strength_home','opp_def_strength_away'], axis = 1)

points.head()

,id_player,full_name,team,position,element,fixture,opponent_team,total_points,was_home,kickoff_time,...,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out,opp_def_strength
0,1,Cédric Alves Soares,Arsenal,DEF,1,1,7,0,False,2022-08-05T19:00:00Z,...,0.00,0.00,0.00,0.00,45,0,23970,0,0,1080
1,1,Cédric Alves Soares,Arsenal,DEF,1,11,10,0,True,2022-08-13T14:00:00Z,...,0.00,0.00,0.00,0.00,44,-5169,24193,1361,6530,1120
2,1,Cédric Alves Soares,Arsenal,DEF,1,21,3,0,False,2022-08-20T16:30:00Z,...,0.00,0.00,0.00,0.00,44,-4337,20960,879,5216,1050
3,1,Cédric Alves Soares,Arsenal,DEF,1,31,9,0,True,2022-08-27T16:30:00Z,...,0.00,0.00,0.00,0.00,43,-2988,18825,577,3565,1130
4,1,Cédric Alves Soares,Arsenal,DEF,1,41,2,0,True,2022-08-31T18:30:00Z,...,0.00,0.00,0.00,0.00,43,-1611,17790,405,2016,1110


In [18]:
# merge opponent attack strength
points = pd.merge(left = points,
                  right = team_strength_att[['id', 'strength_attack_away', 
                                             'strength_attack_home']],
                  how = 'left',
                  left_on = 'opponent_team',
                  right_on = 'id'
).drop(
    'id', axis = 1
).rename(
    columns={'strength_attack_away':'opp_att_strength_away', 
             'strength_attack_home':'opp_att_strength_home'}
)

points.head()

,id_player,full_name,team,position,element,fixture,opponent_team,total_points,was_home,kickoff_time,...,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out,opp_def_strength,opp_att_strength_away,opp_att_strength_home
0,1,Cédric Alves Soares,Arsenal,DEF,1,1,7,0,False,2022-08-05T19:00:00Z,...,0.00,0.00,45,0,23970,0,0,1080,1170,1100
1,1,Cédric Alves Soares,Arsenal,DEF,1,11,10,0,True,2022-08-13T14:00:00Z,...,0.00,0.00,44,-5169,24193,1361,6530,1120,1160,1080
2,1,Cédric Alves Soares,Arsenal,DEF,1,21,3,0,False,2022-08-20T16:30:00Z,...,0.00,0.00,44,-4337,20960,879,5216,1050,1130,1070
3,1,Cédric Alves Soares,Arsenal,DEF,1,31,9,0,True,2022-08-27T16:30:00Z,...,0.00,0.00,43,-2988,18825,577,3565,1130,1070,1070
4,1,Cédric Alves Soares,Arsenal,DEF,1,41,2,0,True,2022-08-31T18:30:00Z,...,0.00,0.00,43,-1611,17790,405,2016,1110,1130,1110


In [19]:
# assign correct home/away opponent attack strength for each fixture
def opp_att_strength(row):
    if row['was_home'] == False:
        return row['opp_att_strength_home']
    elif row['was_home'] == True:
        return row['opp_att_strength_away']
    else:
        return "Unknown"

points['opp_att_strength'] = points.apply(lambda row: opp_att_strength(row), axis = 1)

points = points.drop(['opp_att_strength_home','opp_att_strength_away'], axis = 1)

points.head()

,id_player,full_name,team,position,element,fixture,opponent_team,total_points,was_home,kickoff_time,...,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out,opp_def_strength,opp_att_strength
0,1,Cédric Alves Soares,Arsenal,DEF,1,1,7,0,False,2022-08-05T19:00:00Z,...,0.00,0.00,0.00,45,0,23970,0,0,1080,1100
1,1,Cédric Alves Soares,Arsenal,DEF,1,11,10,0,True,2022-08-13T14:00:00Z,...,0.00,0.00,0.00,44,-5169,24193,1361,6530,1120,1160
2,1,Cédric Alves Soares,Arsenal,DEF,1,21,3,0,False,2022-08-20T16:30:00Z,...,0.00,0.00,0.00,44,-4337,20960,879,5216,1050,1070
3,1,Cédric Alves Soares,Arsenal,DEF,1,31,9,0,True,2022-08-27T16:30:00Z,...,0.00,0.00,0.00,43,-2988,18825,577,3565,1130,1070
4,1,Cédric Alves Soares,Arsenal,DEF,1,41,2,0,True,2022-08-31T18:30:00Z,...,0.00,0.00,0.00,43,-1611,17790,405,2016,1110,1130


In [20]:
# get 20 top scoring players in all positions
gks = points.loc[points['position'] == 'GKP']
defs = points.loc[points['position'] == 'DEF']
mids = points.loc[points['position'] == 'MID']
fwds = points.loc[points['position'] == 'FWD']

top_20_gks = gks.groupby(
    ['element', 'full_name']
).agg(
    {'total_points':'sum'}
).reset_index(
).sort_values(
    'total_points', ascending=False
).head(20)

top_20_defs = defs.groupby(
    ['element', 'full_name']
).agg(
    {'total_points':'sum'}
).reset_index(
).sort_values(
    'total_points', ascending=False
).head(20)

top_20_mids = mids.groupby(
    ['element', 'full_name']
).agg(
    {'total_points':'sum'}
).reset_index(
).sort_values(
    'total_points', ascending=False
).head(20)

top_20_fwds = fwds.groupby(
    ['element', 'full_name']
).agg(
    {'total_points':'sum'}
).reset_index(
).sort_values(
    'total_points', ascending=False
).head(20)

top_20_fwds.head()

,element,full_name,total_points
35,318,Erling Haaland,169
49,427,Harry Kane,140
8,80,Ivan Toney,110
23,210,Aleksandar Mitrović,83
48,411,Che Adams,70


In [21]:
top_20_all_pos = pd.concat([top_20_gks, top_20_defs, top_20_mids, top_20_fwds], axis = 0)

# show the top 10 players by FPL points
top_20_all_pos.sort_values('total_points', ascending=False)[['full_name', 'total_points']].head(10)

,full_name,total_points
35,Erling Haaland,169
122,Kieran Trippier,141
49,Harry Kane,140
3,Martin Ødegaard,117
162,Miguel Almirón Rejala,112
8,Ivan Toney,110
6,Bukayo Saka,109
151,Marcus Rashford,108
138,Kevin De Bruyne,107
131,Mohamed Salah,103


In [22]:
# select columns of interest
points_select = points[['id_player', 'full_name', 'team', 'position',
                        'total_points',
                        'minutes', 'goals_scored', 'assists', 'clean_sheets', 
                        'goals_conceded', 'own_goals',
                        'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index',
                        'expected_goals', 'expected_assists', 'expected_goal_involvements', 
                        'expected_goals_conceded', 'opp_att_strength', 'opp_def_strength']]

In [23]:
points_select['influence'].astype(float)

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
12812    0.0
12813    0.0
12814    0.0
12815    0.0
12816    0.0
Name: influence, Length: 12817, dtype: float64

In [24]:
def last_5_player(df, player_id):
    ''' 
    get the mean stats for a given player_id over the last 5 fixtures
    prior to most recent fixture and the total points from the most 
    recent fixture. 
    
    assume dataframe is sorted from oldest to newest fixtures
    '''
    df = df[df['id_player'] == player_id]
    
    last_5 = df.tail(5)
    
    d = {'name': last_5['full_name'].iloc[0],
         'id': last_5['id_player'].iloc[0],
         'team': last_5['team'].iloc[0],
        'position': last_5['position'].iloc[0],
        'mean_points': last_5['total_points'].mean(),
        'mean_minutes': last_5['minutes'].mean(),
        'mean_goals_scored': last_5['goals_scored'].mean(),
        'mean_assists': last_5['assists'].mean(),
        'mean_clean_sheets': last_5['clean_sheets'].mean(),
        'mean_goals_conceded': last_5['goals_conceded'].mean(),
        'mean_own_goals': last_5['own_goals'].mean(),
        'mean_saves': last_5['saves'].mean(),
        'mean_bonus': last_5['bonus'].mean(),
        'mean_bps': last_5['bps'].mean(),
        'mean_influence': last_5['influence'].astype(float).mean(),
        'mean_creativity': last_5['creativity'].astype(float).mean(),
        'mean_threat': last_5['threat'].astype(float).mean(),
        'mean_ict': last_5['ict_index'].astype(float).mean(),
        'mean_xg': last_5['expected_goals'].astype(float).mean(),
        'mean_xa': last_5['expected_assists'].astype(float).mean(),
        'mean_xgi': last_5['expected_goal_involvements'].astype(float).mean(),
        'mean_xgc': last_5['expected_goals_conceded'].astype(float).mean(),
        'mean_opp_att': last_5['opp_att_strength'].mean(),
        'mean_opp_def': last_5['opp_def_strength'].mean()}
    
    last_5_mean = pd.DataFrame(data = d, index = [0])
    
    return last_5_mean

# test for Bukayo Saka (id = 13)
last_5_player(points_select, 13)

,name,id,team,position,mean_points,mean_minutes,mean_goals_scored,mean_assists,mean_clean_sheets,mean_goals_conceded,...,mean_influence,mean_creativity,mean_threat,mean_ict,mean_xg,mean_xa,mean_xgi,mean_xgc,mean_opp_att,mean_opp_def
0,Bukayo Saka,13,Arsenal,MID,6.6,90.0,0.6,0.2,0.4,1.0,...,35.24,36.62,29.4,10.14,0.284,0.228,0.512,1.034,1166.0,1182.0


In [25]:
def last_5_all(df):
    ''' get last mean stats for all players in df over the last 5 fixtures
    prior to most recent fixture and the total points from the most 
    recent fixture.
    '''
    last_5_all = pd.DataFrame() # empty dataframe
    for p in df['id_player'].unique():
        player_df = last_5_player(df, p)
        last_5_all = pd.concat([last_5_all, player_df])
    return last_5_all

In [26]:
last_5_df = last_5_all(points_select)

last_5_df.head()

,name,id,team,position,mean_points,mean_minutes,mean_goals_scored,mean_assists,mean_clean_sheets,mean_goals_conceded,...,mean_influence,mean_creativity,mean_threat,mean_ict,mean_xg,mean_xa,mean_xgi,mean_xgc,mean_opp_att,mean_opp_def
0,Cédric Alves Soares,1,Arsenal,DEF,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.00,0.0,0.000,0.000,0.000,1166.0,1182.0
0,Rob Holding,5,Arsenal,DEF,0.4,0.8,0.0,0.0,0.0,0.0,...,0.20,0.00,0.0,0.02,0.0,0.000,0.000,0.028,1166.0,1182.0
0,Kieran Tierney,8,Arsenal,DEF,0.6,22.6,0.0,0.0,0.0,0.6,...,2.52,3.42,0.8,0.68,0.0,0.008,0.008,0.420,1166.0,1182.0
0,Benjamin White,10,Arsenal,DEF,3.6,71.8,0.0,0.0,0.4,0.4,...,13.80,9.04,1.6,2.46,0.0,0.080,0.080,0.832,1166.0,1182.0
0,Takehiro Tomiyasu,14,Arsenal,DEF,1.2,18.6,0.0,0.2,0.0,0.6,...,7.68,5.64,1.2,1.44,0.0,0.012,0.012,0.214,1166.0,1182.0


# Apply Models

## Midfield Players

In [27]:
# load pickled model
with open('mid_model_20230112.pkl', 'rb') as file:  
    mid_model = pickle.load(file)

In [28]:
# apply the model
def predict_points(player_data, test_data, model) -> None:
    '''
    apply predictive model to the data
    '''
    prediction = player_data.assign(predicted=model.predict(test_data))
    
    return prediction[['name', 'team', 'position', 
                       'predicted']].sort_values('predicted', ascending=False).head(10).reset_index()

In [31]:
mid_data = last_5_df[last_5_df['position'] == 'MID']

mid_test = mid_data[['mean_ict', 'mean_xgi']]

predict_points(mid_data, mid_test, mid_model)

,index,name,team,position,predicted
0,0,Riyad Mahrez,Man City,MID,3.606422
1,0,Solly March,Brighton,MID,3.517189
2,0,Bukayo Saka,Arsenal,MID,3.430826
3,0,Gabriel Martinelli Silva,Arsenal,MID,3.420604
4,0,James Ward-Prowse,Southampton,MID,3.408169
5,0,Marcus Rashford,Man Utd,MID,3.383193
6,0,Martin Ødegaard,Arsenal,MID,3.353448
7,0,Bruno Borges Fernandes,Man Utd,MID,3.323282
8,0,Jarrod Bowen,West Ham,MID,3.061350
9,0,Dango Ouattara,Bournemouth,MID,3.030526
